In [1]:
import sc2reader
from sc2reader.engine.plugins import APMTracker, ContextLoader, SelectionTracker
from sc2reader import events, data
import pandas as pd
import glob
from IPython.display import display
import json
import pickle
from collections import Counter

pd.options.display.max_columns = None

In [2]:
with open('./stats.json', 'rb') as f:
    stats_attrs = json.load(f)

unit_data = json.loads(data.unit_data)

unit_map = {}
for k in unit_data:
    unit_map[k] = list(unit_data[k].keys())

In [9]:
replays = []
paths = [path for path in glob.glob('./_data/**/*.SC2Replay', recursive=True)]

count = 100

for i, path in enumerate(paths[:count]):
    print('\r%6.2f%% complete' % ((i+1)/count*100), end='', flush=True)
    
    replay = sc2reader.load_replay(path, engine=sc2reader.engine.GameEngine(plugins=[ContextLoader(), APMTracker(), SelectionTracker()]))
    replays.append(replay)

100.00% complete

In [10]:
for event in replays[0].messages:
    print('%s\tmessage="%s"' % (event, event.text))

replays[0].players[0].name
print(replays[0].players[0], replays[0].players[0].result)

00.00	Seither         ChatEvent	message="glhf"
00.10	Probe           ChatEvent	message="gl hf"
14.28	Seither         ChatEvent	message="gg"
Player 1 - Seither (Terran) Loss


In [44]:
df_data = []

valid_games = 0
pt_dict = dict.fromkeys(unit_map['Protoss'] + unit_map['Terran'], 0)

for i, replay in enumerate(replays[:]):
	# print('\r%6.2f%% complete' % ((i+1)/count*100), end='', flush=True)

	if len(set([replay.players[0].pick_race[0], replay.players[1].pick_race[0]]) & set(['P','T'])) == 2:
		print('\n{} Game #{:03} | {} vs. {} {}'.format('-'*17, i+1, replay.players[0].pick_race, replay.players[1].pick_race, '-'*17))

		valid_games += 1
		dd = {}

		for event in replay.events:
			if isinstance(event, events.PlayerLeaveEvent):
				print('Player {} left {} seconds into the game.'.format(event.player, event.second))
				break

			if event.second % 30 == 0:

				# every 10 seconds
				if isinstance(event, events.PlayerStatsEvent):
					d = {}

					is_player_1 = replay.players[1].pid == event.pid
					race = replay.players[is_player_1].pick_race[0]
					win = replay.players[is_player_1].result == 'Win'

					lower_bound = 0 if event.second == 0 else event.second-30
					ap30s = sum(list(replay.players[is_player_1].aps.values())[lower_bound:event.second])

					d['match_id'] = i
					d['frame'] = event.frame
					d['second'] = event.second
					d['race'] = race
					d['ap30s'] = ap30s

					for attr in stats_attrs['PlayerStatsEvent']:
						d[attr] = eval('event.' + attr)
					
					d['win'] = win

					dd[race] = d

				# every 15 seconds
				if isinstance(event, events.UnitPositionsEvent):
					dd1 = dd['P']
					dd2 = dd['T']

					dd1.update(pt_dict)
					dd2.update(pt_dict)

					current_units = [str(a).split(' ')[0].lower() for a in event.units.keys()]
					counted_units = Counter(current_units)

					for k in counted_units:
						if k in unit_map['Protoss']:
							dd1[k] = counted_units[k]
						elif k in unit_map['Terran']:
							dd2[k] = counted_units[k]
						else:
							print('Found invalid unit "{}".'.format(k))

					df_data.extend([dd1, dd2])
					dd = {}

df = pd.DataFrame(df_data)
print('\nEND: ({}, {}) found {} valid games out of {}.'.format(*df.shape, valid_games, len(replays)))


----------------- Game #001 | Terran vs. Protoss -----------------
Player Player 1 - Seither (Terran) left 869 seconds into the game.

----------------- Game #002 | Terran vs. Protoss -----------------
Found invalid unit "observersiegemode".
Found invalid unit "liberatorag".
Found invalid unit "liberatorag".
Player Player 1 - Seither (Terran) left 1620 seconds into the game.

----------------- Game #003 | Protoss vs. Terran -----------------
Found invalid unit "liberatorag".
Player Player 1 - Probe (Protoss) left 978 seconds into the game.

----------------- Game #004 | Protoss vs. Terran -----------------
Player Player 2 - Seither (Terran) left 1697 seconds into the game.

----------------- Game #015 | Terran vs. Protoss -----------------
Player Player 2 - Probe (Protoss) left 779 seconds into the game.

----------------- Game #016 | Terran vs. Protoss -----------------
Player Player 1 - Seither (Terran) left 502 seconds into the game.

----------------- Game #017 | Terran vs. Protos

In [41]:
df.describe()

,match_id,frame,second,ap30s,minerals_current,vespene_current,minerals_collection_rate,vespene_collection_rate,workers_active_count,minerals_used_in_progress,vespene_used_in_progress,minerals_used_current,vespene_used_current,minerals_lost,vespene_lost,minerals_killed,vespene_killed,food_used,food_made,minerals_used_active_forces,vespene_used_active_forces,adept,archon,assimilator,carrier,colossus,cyberneticscore,darkshrine,darktemplar,disruptor,fleetbeacon,forge,gateway,hightemplar,immortal,interceptor,mothership,mothershipcore,nexus,observer,oracle,phoenix,photoncannon,probe,pylon,reactor,roboticsbay,roboticsfacility,sentry,stalker,stargate,tempest,templararchive,twilightcouncil,voidray,warpgate,warpprism,warpprismphasing,zealot,armory,autoturret,banshee,barracks,barrackstechlab,barracksreactor,barracksflying,battlecruiser,battlehellion,bunker,commandcenter,commandcenterflying,cyclone,engineeringbay,factory,factoryflying,factoryreactor,factorytechlab,fusioncore,ghost,ghostacademy,hellion,marauder,marine,medivac,missileturret,mule,orbitalcommand,orbitalcommandflying,planetaryfortress,raven,reaper,refinery,scv,sensortower,siegetank,siegetanksieged,starport,starportflying,starportreactor,starporttechlab,supplydepot,supplydepotlowered,techlab,thor,viking,vikingassault,warhound,widowmine,widowmineburrowed
count,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.00000,716.000000,716.000000,716.0,716.0,716.000000,716.0,716.0,716.000000,716.000000,716.0,716.0,716.0,716.0,716.000000,716.0,716.0,716.0,716.0,716.000000,716.000000,716.000000,716.0,716.000000,716.0,716.0,716.0,716.0,716.000000,716.000000,716.0,716.000000,716.0,716.0,716.000000,716.0,716.000000,716.000000,716.000000,716.0,716.0,716.000000,716.000000,716.0,716.0,716.0,716.000000,716.0,716.0,716.0,716.0,716.000000,716.0,716.000000,716.0,716.0,716.0,716.0,716.000000,716.0,716.000000,716.00000,716.000000,716.000000,716.0,716.000000,716.000000,716.000000,716.000000,716.000000,716.000000,716.0,716.000000,716.0,716.000000,716.000000,716.000000,716.0,716.0,716.0,716.0,716.0,716.0,716.0,716.000000,716.000000,716.0,716.000000,716.000000
mean,44.776536,11739.888268,733.743017,129.431564,318.705307,315.881285,1745.755587,567.363128,46.023743,602.199721,186.277933,8858.379888,1544.448324,3762.268156,1121.698324,3775.069832,1115.991620,97.456704,122.166201,2350.034916,1127.02514,0.071229,0.071229,0.0,0.0,0.127095,0.0,0.0,0.057263,0.065642,0.0,0.0,0.0,0.0,0.036313,0.0,0.0,0.0,0.0,0.004190,0.044693,0.357542,0.0,0.187151,0.0,0.0,0.0,0.0,0.090782,0.755587,0.0,0.036313,0.0,0.0,0.006983,0.0,0.009777,0.005587,0.506983,0.0,0.0,0.011173,0.002793,0.0,0.0,0.0,0.001397,0.0,0.0,0.0,0.0,0.034916,0.0,0.001397,0.0,0.0,0.0,0.0,0.108939,0.0,0.022346,0.97486,1.828212,0.124302,0.0,0.002793,0.011173,0.004190,0.005587,0.005587,0.048883,0.0,0.248603,0.0,0.092179,0.173184,0.002793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111732,0.011173,0.0,0.004190,0.032123
std,41.331616,6155.845478,384.740342,59.126582,259.086542,292.174014,665.285389,284.897694,17.360618,335.573636,155.811908,4596.769878,1179.414669,5002.404931,1667.245038,5018.866155,1659.305987,50.631559,64.070560,1798.586517,934.07395,0.383884,0.412001,0.0,0.0,0.483915,0.0,0.0,0.359977,0.358542,0.0,0.0,0.0,0.0,0.261934,0.0,0.0,0.0,0.0,0.064639,0.238204,1.644006,0.0,0.739421,0.0,0.0,0.0,0.0,0.437889,1.871474,0.0,0.486253,0.0,0.0,0.098698,0.0,0.098461,0.074587,1.626255,0.0,0.0,0.129067,0.052815,0.0,0.0,0.0,0.037372,0.0,0.0,0.0,0.0,0.191158,0.0,0.037372,0.0,0.0,0.0,0.0,0.512095,0.0,0.189377,2.49294,3.797657,0.478835,0.0,0.052815,0.105185,0.064639,0.074587,0.074587,0.234414,0.0,0.986446,0.0,0.554595,0.630823,0.052815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500435,0.117733,0.0,0.064639,0.275481
min,0.000000,1920.000000,120.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,1475.00

In [49]:
df.to_pickle('./sc2_{}_{}{}.pickle'.format(valid_games, 'P', 'T'))
df.to_csv('./sc2_{}_{}{}.csv'.format(valid_games, 'P', 'T'))